In [1]:
import json
import gzip

In [1]:
import joblib

file_name_list = joblib.load('./fasion_data/train_file_name_list_08_04')
class_label_list = joblib.load('./fasion_data/train_class_label_list_08_04')

In [3]:
class_label_list

[14,
 2,
 4,
 26,
 15,
 4,
 19,
 11,
 23,
 7,
 24,
 11,
 28,
 15,
 9,
 2,
 8,
 20,
 6,
 31,
 12,
 11,
 26,
 30,
 26,
 31,
 17,
 3,
 11,
 28,
 13,
 1,
 12,
 19,
 2,
 25,
 12,
 24,
 5,
 17,
 12,
 21,
 25,
 16,
 9,
 31,
 11,
 32,
 27,
 28,
 2,
 28,
 13,
 24,
 32,
 7,
 5,
 4,
 5,
 6,
 18,
 14,
 3,
 6,
 6,
 30,
 0,
 6,
 9,
 26,
 28,
 13,
 28,
 7,
 2,
 14,
 10,
 8,
 8,
 4,
 28,
 22,
 18,
 30,
 9,
 26,
 3,
 19,
 8,
 4,
 1,
 15,
 26,
 25,
 23,
 15,
 19,
 26,
 4,
 19,
 9,
 16,
 19,
 32,
 5,
 11,
 23,
 14,
 24,
 4,
 23,
 14,
 12,
 30,
 31,
 4,
 21,
 2,
 12,
 30,
 5,
 13,
 31,
 24,
 14,
 11,
 22,
 19,
 12,
 5,
 15,
 11,
 18,
 25,
 13,
 4,
 21,
 4,
 22,
 7,
 0,
 26,
 30,
 17,
 27,
 22,
 4,
 27,
 13,
 27,
 5,
 15,
 15,
 28,
 30,
 24,
 19,
 30,
 8,
 24,
 31,
 24,
 6,
 10,
 31,
 5,
 29,
 29,
 4,
 9,
 15,
 32,
 32,
 23,
 19,
 7,
 22,
 12,
 21,
 27,
 24,
 6,
 1,
 2,
 0,
 27,
 15,
 29,
 15,
 20,
 7,
 23,
 15,
 30,
 9,
 29,
 11,
 14,
 12,
 2,
 1,
 12,
 18,
 21,
 26,
 8,
 31,
 27,
 13,
 2,
 20,
 31,
 4,
 

In [2]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [3]:
line_list = []

mis_count = 0
for d in parse('./fasion_data/metadata.json.gz'):
    try:
        if 'Clothing' in d['categories'][0][0]:
            line_list.append(d)
    except Exception as e:
        mis_count += 1
        continue

In [7]:
len(line_list)

1435869

In [8]:
df = pd.DataFrame(line_list)

In [18]:
df.to_pickle('./fasion_data/fashion_df')

## Grouping by categories

## 카테고리

### 1단계 : Clothing, Shoes & Jewelry

### 2단계 : Women || Men || Girls || Boys

### 3단계 : Clothing

In [12]:
grouped_df = df.groupby('categories')

## 카테고리 필터링

### 제외 : 3단계 까지의 카테고리

### 4단계 카테고리 까지만

### [ 4단계 기준 ] 카테고리 44개, 데이터 265742개

In [78]:
cate_dict = {}
filter_list = []

cate_1 = 'Clothing, Shoes & Jewelry'
cate_2_list = ['Women', 'Men', 'Girls', 'Boys']
cate_3_list = ['Clothing']

for each in df.iterrows():
    cate_length = len(each[1]['categories'][0])
    
    if cate_length < 5:
        continue
    
    cate_2 = each[1]['categories'][0][1]
    cate_3 = each[1]['categories'][0][2]
    cate_4 = each[1]['categories'][0][3]
    
    if cate_3 in cate_3_list:
        filter_list.append(each[1])
        cate = cate_1 + '||' + cate_2 + '||' + cate_3 + '||' + cate_4
        if cate in cate_dict:    
            cate_dict[cate] += 1
        else:
            cate_dict[cate] = 0

In [79]:
cate_dict

{'Clothing, Shoes & Jewelry||Boys||Clothing||Active': 436,
 'Clothing, Shoes & Jewelry||Boys||Clothing||Button-Down & Dress Shirts': 586,
 'Clothing, Shoes & Jewelry||Boys||Clothing||Clothing Sets': 1457,
 'Clothing, Shoes & Jewelry||Boys||Clothing||Jackets & Coats': 1469,
 'Clothing, Shoes & Jewelry||Boys||Clothing||Sleepwear & Robes': 1896,
 'Clothing, Shoes & Jewelry||Boys||Clothing||Socks': 225,
 'Clothing, Shoes & Jewelry||Boys||Clothing||Suits & Sport Coats': 677,
 'Clothing, Shoes & Jewelry||Boys||Clothing||Sweaters': 242,
 'Clothing, Shoes & Jewelry||Boys||Clothing||Swim': 976,
 'Clothing, Shoes & Jewelry||Boys||Clothing||Tops & Tees': 3460,
 'Clothing, Shoes & Jewelry||Boys||Clothing||Underwear': 547,
 'Clothing, Shoes & Jewelry||Girls||Clothing||Active': 213,
 'Clothing, Shoes & Jewelry||Girls||Clothing||Clothing Sets': 3039,
 'Clothing, Shoes & Jewelry||Girls||Clothing||Dresses': 8773,
 'Clothing, Shoes & Jewelry||Girls||Clothing||Jackets & Coats': 1631,
 'Clothing, Shoes & 

In [80]:
len(filter_list)

265742

## Download Images

In [10]:
import wget
import pandas as pd
from pathlib import Path

In [85]:
filtered_df = pd.DataFrame(filter_list)

In [94]:
len(filtered_df)

265742

In [86]:
filtered_df.to_pickle('./fasion_data/filtered_df')

In [11]:
filtered_df = pd.read_pickle('./fasion_data/filtered_df')

In [18]:
count = 0
not_count = 0
dir_num = 0
for each in filtered_df.iterrows():
    each = each[1]
    if count % 30000 == 0:
        dir_num += 1
    f_name = 'dir_' + str(dir_num) + '/' + each['asin']
    try:
        file_name = wget.download(each['imUrl'], out= f_name)
        count += 1
    except Exception as e:
        print(e)
        not_count += 1

'float' object has no attribute 'find'
'float' object has no attribute 'find'
'float' object has no attribute 'find'


## Verify if the files are image file 
### if non verified file then remove

In [6]:
import imghdr
import os

In [49]:
file_list = (glob.glob('dir_9/*'))

In [50]:
count = 0
remove_file_list = []
for file_name in file_list:
    if (imghdr.what(file_name)) != 'jpeg':
        remove_file_list.append(file_name)
        count += 1

In [52]:
remove_file_list[1]

'dir_9/B00H4UM3YK'

In [53]:
for i in range(len(remove_file_list)):
    os.remove(remove_file_list[i])

## Drop category if has less then 1000 data files

In [2]:
import joblib

filtered_df = joblib.load('./fasion_data/filtered_df')

In [3]:
filtered_df

,asin,brand,categories,description,imUrl,price,related,salesRank,title,cate_1_4
1,0000031887,Boutique Cutie,"[[Clothing, Shoes & Jewelry, Girls, Clothing, ...",This adorable basic ballerina tutu is perfect ...,http://ecx.images-amazon.com/images/I/314qZjYe...,6.79,"{u'also_bought': [u'0000031852', u'0000031895'...",{u'Sports &amp; Outdoors': 8547},Ballet Dress-Up Fairy Tutu,"Clothing, Shoes & Jewelry||Girls||Clothing||Ac..."
74,245600258X,NaN,"[[Clothing, Shoes & Jewelry, Men, Clothing, Sh...",NaN,http://ecx.images-amazon.com/images/I/515a0tzH...,NaN,"{u'also_bought': [u'B00CVT2ZRO', u'B00CVT3CYO'...",{u'Clothing': 3575},La Leela Men's Hawaiian Print Shirt,"Clothing, Shoes & Jewelry||Men||Clothing||Shirts"
77,3999837810,NaN,"[[Clothing, Shoes & Jewelry, Men, Clothing, Pa...",NaN,http://ecx.images-amazon.com/images/I/41nxMluc...,179.00,"{u'also_viewed': [u'B00EPHEZJG', u'B001AL7ONO'...",{u'Clothing': 138136},Fuente Deluxe Aniline Leather Pants black for Men,"Clothing, Shoes & Jewelry||Men||Clothing||Pants"
80,3999838167,NaN,"[[Clothing, Shoes & Jewelry, Men, Clothing, Pa...",NaN,http://ecx.images-amazon.com/images/I/41rnYT7w...,145.00,"{u'also_viewed': [u'B00CZ8YX8K', u'B00CYG0M2E'...",{u'Clothing': 1302944},Fuente Lion Brown Leather Pants trousers Germany,"Clothing, Shoes & Jewelry||Men||Clothing||Pants"
82,481713996X,NaN,"[[Clothing, Shoes & Jewelry, Men, Clothing, Pa...",NaN,http://ecx.images-amazon.com/images/I/41MEM6xw...,229.00,"{u'also_viewed': [u'B00CYG0M2E', u'B00GWQD6MK'...",{u'Clothing': 479034},Leather Trousers Bockle Men's Saddle Lederjean...,"Clothing, Shoes & Jewelry||Men||Clothing||Pants"
125,7890004311,NaN,"[[Clothing, Shoes & Jewelry, Men, Clothing, Ja...",NaN,http://ecx.images-amazon.com/images/I/41Q-edGa...,149.99,"{u'also_viewed': [u'B00AOUVWSK', u'B00ECNXF0S'...",{u'Clothing': 598476},Emaan's Men Faux Leather Bane Vest,"Clothing, Shoes & Jewelry||Men||Clothing||Jack..."
168,9629558424,NaN,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...","Packages include : 6Pcs - 2 for each color, 12...",http://ecx.images-amazon.com/images/I/511Kl0i9...,6.38,"{u'also_viewed': [u'B002KABQR4', u'B00F2ZCPTW'...",{u'Health & Personal Care': 487779},Bra Strap Extenders 2X3 and 3X3 HOOK-White bei...,"Clothing, Shoes & Jewelry||Women||Clothing||Li..."
170,978979262X,NaN,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",NaN,http://ecx.images-amazon.com/images/I/41p5NTR5...,NaN,NaN,{u'Clothing': 2214183},RACER CORSET - SMALL,"Clothing, Shoes & Jewelry||Women||Clothing||Li..."
183,978980685X,NaN,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",NaN,http://ecx.images-amazon.com/images/I/41mbEuL5...,NaN,NaN,NaN,OFF THE SHOULDER DIAMOND NET MINI DRESS - BLAC...,"Clothing, Shoes & Jewelry||Women||Clothing||Li..."
337,B000052XTI,NaN,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",NaN,http://ecx.images-amazon.com/images/I/51%2Bxen...,NaN,"{u'also_viewed': [u'B0057D80FM', u'B00B994NYK'...",NaN,"Futuro Beyond Support Ultra Sheer Knee High, M...","Clothing, Shoes & Jewelry||Women||Clothing||So..."


In [2]:
import pandas as pd

c = 0

cate_filter_list = []
for each in filtered_df.groupby('cate_1_4'):
    if len(each[1]) > 1000:
        cate_count = 0
        for data in each[1].iterrows():
            if cate_count > 2000:
                break
            cate_filter_list.append(data[1])
            cate_count += 1
cate_filtered_df = pd.DataFrame(cate_filter_list)
cate_filtered_df.to_pickle('./fasion_data/cate_filtered_df_08_04')

In [5]:
cate_dict = {}
c = 0
for each in cate_filtered_df.groupby('cate_1_4'):
    print(each[0], len(each[1]))
    cate_dict[each[0]] = c
    c += 1

('Clothing, Shoes & Jewelry||Boys||Clothing||Clothing Sets', 1458)
('Clothing, Shoes & Jewelry||Boys||Clothing||Jackets & Coats', 1470)
('Clothing, Shoes & Jewelry||Boys||Clothing||Sleepwear & Robes', 1897)
('Clothing, Shoes & Jewelry||Boys||Clothing||Tops & Tees', 2001)
('Clothing, Shoes & Jewelry||Girls||Clothing||Clothing Sets', 2001)
('Clothing, Shoes & Jewelry||Girls||Clothing||Dresses', 2001)
('Clothing, Shoes & Jewelry||Girls||Clothing||Jackets & Coats', 1632)
('Clothing, Shoes & Jewelry||Girls||Clothing||Sleepwear & Robes', 2001)
('Clothing, Shoes & Jewelry||Girls||Clothing||Swim', 1702)
('Clothing, Shoes & Jewelry||Girls||Clothing||Tops & Tees', 2001)
('Clothing, Shoes & Jewelry||Men||Clothing||Active', 2001)
('Clothing, Shoes & Jewelry||Men||Clothing||Jackets & Coats', 2001)
('Clothing, Shoes & Jewelry||Men||Clothing||Pants', 2001)
('Clothing, Shoes & Jewelry||Men||Clothing||Shirts', 2001)
('Clothing, Shoes & Jewelry||Men||Clothing||Shorts', 2001)
('Clothing, Shoes & Jewelry|

In [8]:
joblib.dump(cate_dict, 'fasion_data/filtered_cate_mapping_dict_08_04')

['fasion_data/filtered_cate_mapping_dict_08_04']

In [22]:
len(cate_dict)

33

In [12]:
import glob
file_list = (glob.glob('amazon_train_img/*'))

file_name_list = []
class_label_list = []

c = 0
for file_name in file_list:
    try:
        f_name = file_name.split('/')[1]
        class_label = cate_dict[cate_filtered_df[cate_filtered_df['asin'] == f_name]['cate_1_4'].values[0]]
        file_name_list.append(file_name)
        class_label_list.append(class_label)
    except Exception as e:
        c += 1

In [15]:
joblib.dump(file_name_list, './fasion_data/train_file_name_list_08_04')
joblib.dump(class_label_list, './fasion_data/train_class_label_list_08_04')

['./fasion_data/train_class_label_list_08_04']

In [65]:
file_name_list = []
class_label_list = []

for file_name in file_list:
    try:
        f_name = file_name.split('/')[1]
        class_label = cate_dict[filtered_df[filtered_df['asin'] == f_name]['cate_1_4'].values[0]]

        file_name_list.append(file_name)
        class_label_list.append(class_label)
    except Exception as e:
        print(e)
        print(file_name)
        print('\n')

index 0 is out of bounds for axis 0 with size 0
amazon_train_img/B003LL1XH4LLCdgl.tmp


index 0 is out of bounds for axis 0 with size 0
amazon_train_img/B003LL1XH4X0XeGz.tmp


index 0 is out of bounds for axis 0 with size 0
amazon_train_img/B003LL1XH4f8odig.tmp


index 0 is out of bounds for axis 0 with size 0
amazon_train_img/B003LL1XH4M0XMe2.tmp


index 0 is out of bounds for axis 0 with size 0
amazon_train_img/B003LL1XH4_j2y88.tmp


index 0 is out of bounds for axis 0 with size 0
amazon_train_img/B003LL1XH4Hnwges.tmp


index 0 is out of bounds for axis 0 with size 0
amazon_train_img/B003LL1XH4QntBEf.tmp


index 0 is out of bounds for axis 0 with size 0
amazon_train_img/B003LL1XH4xcbDwp.tmp


index 0 is out of bounds for axis 0 with size 0
amazon_train_img/B003LL1XH4cRTqLX.tmp


index 0 is out of bounds for axis 0 with size 0
amazon_train_img/B003LL1XH4jRylvN.tmp


index 0 is out of bounds for axis 0 with size 0
amazon_train_img/B003LL1XH4cb3Xw6.tmp


index 0 is out of bounds for axi

In [66]:
joblib.dump(file_name_list, './fasion_data/train_file_name_list')
joblib.dump(class_label_list, './fasion_data/train_class_label_list')

['./fasion_data/train_class_label_list']

In [36]:
import joblib

file_list = joblib.load('./fasion_data/train_file_name_list')

In [11]:
print(file_name_list[0:10])
print(class_label_list[0:10])

['amazon_train_img/B008I5834O', 'amazon_train_img/B00804W4A2', 'amazon_train_img/B0081L62XO', 'amazon_train_img/B00B659GYE', 'amazon_train_img/B00E4H80NY', 'amazon_train_img/B009ERED6C', 'amazon_train_img/B00EKPR9HI', 'amazon_train_img/B00DIW9BQQ', 'amazon_train_img/B000PVV4P4', 'amazon_train_img/B009JLWPJA']
[16, 3, 34, 1, 31, 5, 23, 43, 4, 1]


## Create hash table to map file name list with class label list for trining

In [4]:
table = tf.contrib.lookup.HashTable(
        tf.contrib.lookup.KeyValueTensorInitializer(file_name_list, class_label_list, key_dtype=tf.string, value_dtype=tf.int32), -1)

## Set image size to 128 and do some processing stuff for training

In [14]:
image_size = 128

filename_queue = tf.train.string_input_producer(tf.gfile.Glob("amazon_train_img/B0081L62XO"))
image_reader = tf.WholeFileReader()

file_path, image_file = image_reader.read(filename_queue)
image = tf.image.decode_jpeg(image_file, channels=3)
longer = tf.reduce_max(tf.shape(image))
image = tf.image.resize_image_with_crop_or_pad(image, longer, longer)
image = tf.image.resize_images(image, [128, 128], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

### (128, 32, 32, 3)
### (128, 10)


In [41]:
amazon_data_batch = open('./fasion_data/amazon_data_batch_1.bin', 'wb')

count = 0

for file_name in file_list:
    # In this case f_name == asin 
    print(file_name)
    f_name = file_name.split('/')[1]
#     class_label = cate_dict[filtered_df[filtered_df['asin'] == f_name]['cate_1_4'].values[0]]
#     class_label_tensor = tf.cast([class_label], tf.int32)
    
#     print(f_name, class_label, count)
    

    file_name_queue = tf.train.string_input_producer(tf.gfile.Glob(file_name))
    image_reader = tf.WholeFileReader()
    _, image_file = image_reader.read(file_name_queue)
    image = tf.image.decode_jpeg(image_file, channels=3)
    print(image.shape)
    image = tf.image.resize_image_with_crop_or_pad(image, 256, 256)

    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        print(image.shape)
#         print(class_label_tensor.shape)
        sess.close()
    break

amazon_train_img/B008I5834O
(?, ?, 3)
(256, 256, 3)


In [6]:
import tensorflow as tf

In [29]:
a = tf.placeholder(tf.float32, shape=[1, 128,128,3])

In [35]:
tf.shape(tf.squeeze(a, [0]))

<tf.Tensor 'Shape_6:0' shape=(3,) dtype=int32>

In [30]:
if tf.shape(a) == (4,):
    print('a')

In [34]:
len(a.shape.as_list())

4